# 字段说明
——表1 user.csv：

- user_id 用户id
- register_time 注册时间
- recently_logged 最近访问时间
- learn_time 学习时间（分）
- number_of_classes_join 加入班级数
- number_of_classes_out 退出班级数
- school 用户所属学校

——表 2 study_information.csv 字段说明：

- user_id 用户 id
- course_id 课程 id
- course_join_time 加入课程的时间
- learn_process 学习进度
- price 课程单价

——表 3 login.csv 字段说明：

- 字段名 描述
- user_id 用户 id
- login_time 登录时间
- login_place 登录地址

# 任务一：数据预处理
- 任务 1.1 对照附录1，理解各字段的含义，进行缺失值、重复值等方面的必12要处理，将处理结果保存为“task1_1_X.csv”（如果包含多张数据表，X 可从 1 开始往后编号），并在报告中描述处理过程
- 任务 1.2 对用户信息表中 recently_logged 字段的“--”值进行必要的处理，将处理结果保存为“task1_2.csv”，并在报告中描述处理过程

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all' #默认为'last'

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import datetime
import jieba
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
login = pd.read_csv('./data/login.csv',encoding='gbk')
stu_info = pd.read_csv('./data/study_information.csv',encoding='gbk')
users = pd.read_csv('./data/users.csv',encoding='gbk')

## login

### login表缺失值

首先判断该缺失值是否为真实缺失值。针对不同的数据缺失情况，本次分析将会采用不同的处理方式：

- 针对数据为0的情况，需要进行实际的分析，回归到原始数据中去，判断该数据为0时是否具有实际意义。如果没有实际意义就将其作为缺失值处理
- 针对数据为null的情况，如果该特征数据缺失情况低于10%，则结合该特征的重要性进行综合判断。如果该字段的重要性比较低，则考虑直接删除，如果该字段的重要性比较高，则考虑利用插值或者均值填充进行填补

In [3]:
login.head()

,user_id,login_time,login_place
0,用户3,2018-09-06 09:32:47,中国广东广州
1,用户3,2018-09-07 09:28:28,中国广东广州
2,用户3,2018-09-07 09:57:44,中国广东广州
3,用户3,2018-09-07 10:55:07,中国广东广州
4,用户3,2018-09-07 12:28:42,中国广东广州


In [4]:
login.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387144 entries, 0 to 387143
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      387144 non-null  object
 1   login_time   387144 non-null  object
 2   login_place  387144 non-null  object
dtypes: object(3)
memory usage: 8.9+ MB


- **结论** : 初步观察login表无明显缺失值

### login表异常值

In [5]:
# for i in login.columns:
#     a = login[i]  
#     print(a.value_counts().sort_values()[-10:]) 

- **结论** ：无明显异常值

### login表重复值

In [6]:
login.drop_duplicates(inplace=True)

## stu_info

### stu_info表缺失值

In [7]:
stu_info.head()

,user_id,course_id,course_join_time,learn_process,price
0,用户3,课程106,2020-04-21 10:11:50,width: 0%;,0.0
1,用户3,课程136,2020-03-05 11:44:36,width: 1%;,0.0
2,用户3,课程205,2018-09-10 18:17:01,width: 63%;,0.0
3,用户4,课程26,2020-03-31 10:52:51,width: 0%;,319.0
4,用户4,课程34,2020-03-31 10:52:49,width: 0%;,299.0


In [8]:
stu_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194974 entries, 0 to 194973
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           194974 non-null  object 
 1   course_id         194974 non-null  object 
 2   course_join_time  194974 non-null  object 
 3   learn_process     194974 non-null  object 
 4   price             190736 non-null  float64
dtypes: float64(1), object(4)
memory usage: 7.4+ MB


- price列存在缺失数据

In [9]:
stu_info[stu_info['price'].isnull()]['course_id'].value_counts()

课程51    4011
课程96     227
Name: course_id, dtype: int64

- 查看同一课程是否存在差异定价

In [10]:
stu_course = stu_info.groupby(['course_id']).agg({'price':['max','min']})
stu_course

price       
             max    min
course_id              
课程0        199.0  199.0
课程1        199.0  199.0
课程10         0.0    0.0
课程100      199.0  199.0
课程101        0.0    0.0
...          ...    ...
课程95       499.0  499.0
课程96         NaN    NaN
课程97        29.0   29.0
课程98        99.0   99.0
课程99        59.0   59.0

[241 rows x 2 columns]

In [11]:
stu_course[stu_course['price']['max']-stu_course['price']['min'] != 0]

price    
            max min
course_id          
课程51        NaN NaN
课程96        NaN NaN

- **结论**：不存在差异定价的现象，其中课程51和课程96共计4238条数据，暂时不做处理

### stu_info表异常值

In [12]:
# for i in stu_info.columns[:-1]:
#     a = stu_info[i]  
#     print(a.value_counts().sort_values()[-10:]) 

- **结论** ：没有明显的异常值

### stu_info重复值处理

In [13]:
stu_info = stu_info.drop_duplicates()
stu_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194974 entries, 0 to 194973
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           194974 non-null  object 
 1   course_id         194974 non-null  object 
 2   course_join_time  194974 non-null  object 
 3   learn_process     194974 non-null  object 
 4   price             190736 non-null  float64
dtypes: float64(1), object(4)
memory usage: 8.9+ MB


- stu_info没有完全重复的行

In [14]:
stu_info.to_csv('./part01/study_information.csv',index=False)

## users

### users缺失值处理

In [15]:
users.head()

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school
0,用户44251,2020/6/18 9:49,2020/6/18 9:49,0,0,41.25,NaN
1,用户44250,2020/6/18 9:47,2020/6/18 9:48,0,0,0,NaN
2,用户44249,2020/6/18 9:43,2020/6/18 9:43,0,0,16.22,NaN
3,用户44248,2020/6/18 9:09,2020/6/18 9:09,0,0,0,NaN
4,用户44247,2020/6/18 7:41,2020/6/18 8:15,0,0,1.8,NaN


In [16]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43983 entries, 0 to 43982
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   user_id                 43916 non-null  object
 1   register_time           43983 non-null  object
 2   recently_logged         43983 non-null  object
 3   number_of_classes_join  43983 non-null  int64 
 4   number_of_classes_out   43983 non-null  int64 
 5   learn_time              43983 non-null  object
 6   school                  10571 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.3+ MB


- 考虑到user_id是用户的唯一标识，为not null特征。缺失数据为67行，相对于43983的数据量相对较少,直接删除

In [17]:
users = users[~users['user_id'].isnull()]

In [18]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43916 entries, 0 to 43982
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   user_id                 43916 non-null  object
 1   register_time           43916 non-null  object
 2   recently_logged         43916 non-null  object
 3   number_of_classes_join  43916 non-null  int64 
 4   number_of_classes_out   43916 non-null  int64 
 5   learn_time              43916 non-null  object
 6   school                  10569 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.7+ MB


- school字段严重缺失，考虑实际情况，为选填选项。暂时保留，并且创建是否填写学校信息

In [19]:
users['school_info'] = users['school']
users['school_info'][~users['school_info'].isnull()] = 1
users['school_info'][users['school_info'].isnull()] = 0

In [20]:
users['school_info'].value_counts()

0    33347
1    10569
Name: school_info, dtype: int64

### users异常值处理

In [21]:
# for i in users.columns:
#     a = users[i]  
#     print(a.value_counts().sort_values(ascending=False)[:3]) 

In [22]:
miss_logged = users[users['recently_logged']=='--']
miss_logged

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,school_info
11,用户44240,2020/6/17 17:25,--,1,0,"1,667.28",NaN,0
12,用户44239,2020/6/17 17:24,--,1,0,"2,109.75",NaN,0
14,用户44235,2020/6/17 16:39,--,1,0,0,NaN,0
15,用户44237,2020/6/17 16:39,--,1,0,"10,348.62",NaN,0
16,用户44232,2020/6/17 16:39,--,1,0,"9,054.72",NaN,0
...,...,...,...,...,...,...,...,...
43772,用户214,2018/10/25 20:46,--,0,0,0,NaN,0
43789,用户197,2018/10/25 19:53,--,0,0,3.1,NaN,0
43834,用户151,2018/10/25 18:26,--,0,0,0,NaN,0
43868,用户117,2018/10/25 17:47,--,0,0,0,NaN,0


- users和其余表的关联性，异常值分类为：
    - 为流失用户所产生，因此最近登录时间为'--'
    - 用户未流失，属于某一课程记录出错的现象

In [23]:
error_logged = miss_logged.groupby('user_id').count()['register_time']
error_logged[error_logged > 1]

user_id
用户40732    2
Name: register_time, dtype: int64

**结论**
- 数异常数据中-皆独立关联一个用户，仅有用户40732存在两条异常数据
- 因此认为该异常数据为为流失用户产生，暂时不做处理

### users重复值处理

In [24]:
users[users.duplicated(keep=False)]

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,school_info
3428,用户40732,2020/5/25 18:01,--,1,0,0,NaN,0
3429,用户40732,2020/5/25 18:01,--,1,0,0,NaN,0
3534,用户36488,2020/5/22 15:16,2020/6/17 8:25,1,0,"1,006.88",NaN,0
3535,用户36488,2020/5/22 15:16,2020/6/17 8:25,1,0,"1,006.88",NaN,0
36097,用户7909,2019/4/9 15:08,2019/5/1 8:14,0,0,96.85,NaN,0
36098,用户7909,2019/4/9 15:08,2019/5/1 8:14,0,0,96.85,NaN,0


- 其中用户40732的'recently_logged'为'--'
- users记录该用户最近登录的信息，应该把重复值去除

In [25]:
users.drop_duplicates(inplace=True)

In [26]:
users.to_csv('./part01/users.csv',index=False)

## 进一步提取login信息
- 拆分login_place字段，获取国家、省份、地区字段

In [27]:
login_data = login.copy() 
for i in range(login_data.shape[0]):
    if login_data.loc[i,'login_place'][0:2] == '中国':
        login_data.loc[i,'国家'] = '中国'
        if '黑龙江' in login_data.loc[i,'login_place']:
            login_data.loc[i,'省份'] = '黑龙江'
            if len(login_data.loc[i,'login_place']) > 5:
                login_data.loc[i,'地区'] = login_data.loc[i,'login_place'][5:]
                
        if '新疆维吾尔' in login_data.loc[i,'login_place']:
            login_data.loc[i,'省份'] = '新疆维吾尔'
            if len(login_data.loc[i,'login_place']) > 7:
                login_data.loc[i,'地区'] = login_data.loc[i,'login_place'][7:]
                
        if '内蒙古' in login_data.loc[i,'login_place']:
            login_data.loc[i,'省份'] = '内蒙古'
            if len(login_data.loc[i,'login_place']) > 5:
                login_data.loc[i,'地区'] = login_data.loc[i,'login_place'][5:]
                
        else:
            login_data.loc[i,'省份'] = login_data.loc[i,'login_place'][2:4]
            login_data.loc[i,'地区'] = login_data.loc[i,'login_place'][4:]   
            
    else:
        li = [word for word in jieba.cut(login_data.iloc[i,2])]
        if len(li) == 2:
            login_data.loc[i,'国家'] = li[0]
            login_data.loc[i,'省份'] = li[1]
        else:
            login_data.loc[i,'国家'] = li[0]
    
    if i % 10000 == 0:
        print(f'{round(i*100/(int(login_data.shape[0])),2)}%')

0.0%


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zmqmo\AppData\Local\Temp\jieba.cache
Loading model cost 0.622 seconds.
Prefix dict has been built successfully.


2.58%
5.17%
7.75%
10.33%
12.92%
15.5%
18.08%
20.66%
23.25%
25.83%
28.41%
31.0%
33.58%
36.16%
38.75%
41.33%
43.91%
46.49%
49.08%
51.66%
54.24%
56.83%
59.41%
61.99%
64.58%
67.16%
69.74%
72.32%
74.91%
77.49%
80.07%
82.66%
85.24%
87.82%
90.41%
92.99%
95.57%
98.15%


In [28]:
login_data.to_csv('./part01/login_area_split.csv',index=False)